In [1]:
import random
import numpy as np
import torch
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from sklearn.model_selection._split import _RepeatedSplits
from skorch.callbacks import Callback
from skorch.dataset import CVSplit, get_len
from skorch.utils import to_numpy
from torch import nn
from lifelines.utils import concordance_index
from torch import nn

In [2]:
# Adapted from Tong, Li, et al. "Deep learning based feature-level integration of multi-omics data for breast cancer patients survival analysis." BMC medical informatics and decision making 20.1 (2020): 1-12.
# https://github.com/tongli1210/BreastCancerSurvivalIntegration/blob/master/src/models/loss_survival.py
def get_R_matrix_moded_dave(survival_time):
    """
    Create an indicator matrix of risk sets, where T_j >= T_i.

    Input:
        survival_time: a Pytorch tensor that the number of rows is equal top the number of samples
    Output:
        indicator matrix: an indicator matrix
    """
    surv_time = np.array(survival_time)
    return (np.outer(surv_time, surv_time) >= np.transpose([np.square(survival_time)] * survival_time.shape[0])).astype(int).T

def get_R_matrix_moded(survival_time):
    """
    Create an indicator matrix of risk sets, where T_j >= T_i.

    Input:
        survival_time: a Pytorch tensor that the number of rows is equal top the number of samples
    Output:
        indicator matrix: an indicator matrix
    """
    surv_time = np.array(survival_time)
    return (np.outer(surv_time, surv_time) >= np.square(surv_time)).astype(int).T

# Adapted from Tong, Li, et al. "Deep learning based feature-level integration of multi-omics data for breast cancer patients survival analysis." BMC medical informatics and decision making 20.1 (2020): 1-12.
# https://github.com/tongli1210/BreastCancerSurvivalIntegration/blob/master/src/models/loss_survival.py
def neg_par_log_likelihood(
    pred,
    survival_time,
    survival_event,
    sample_weight,
    cuda=False,
):
    """
    Calculate the average Cox negative partial log-likelihood
    Input:
        pred: linear predictors from trained model.
        survival_time: survival time from ground truth
        survival_event: survival event from ground truth: 1 for event
                and 0 for censored
    Output:
        cost: the survival cost to be minimized
    """
    sample_weight = torch.unsqueeze(sample_weight, 1)
    survival_event = torch.tensor(survival_event)
    survival_time = torch.tensor(survival_time)
    n_observed = survival_event.sum(0)
    if not n_observed:
        # Return zero loss if there are no events
        # within a batch.
        return torch.tensor(0.0)
    R_matrix = get_R_matrix_moded(survival_time)
    R_matrix = torch.Tensor(R_matrix)
    if cuda:
        R_matrix = R_matrix.cuda()
    risk_set_sum = R_matrix.mm(torch.exp(pred))
    diff = pred - torch.log(risk_set_sum)
    survival_event = torch.reshape(
        survival_event, (survival_event.shape[0], 1)
    )
    sum_diff_in_observed = (
        torch.transpose(diff * sample_weight, 0, 1).float().mm(survival_event)
    )
    loss = (-(sum_diff_in_observed) / n_observed).reshape((-1,))
    return loss

In [3]:
class ae_criterion(nn.Module):
    def forward(self, prediction, target, sample_weight=None):
        if not sample_weight:
            sample_weight = torch.ones(prediction[0].shape[0])
        mse = nn.MSELoss()
        return neg_par_log_likelihood(
                prediction[0],
                np.array([str.rsplit(i, "|")[1] for i in target]).astype(
                    np.float32
                ),
                np.array([str.rsplit(i, "|")[0] for i in target]).astype(
                    np.float32
                ),
                sample_weight,
        ) + mse(prediction[1], prediction[2])

In [4]:
class Encoder(nn.Module):
    """Base class modeling the encoder portion of an autoencoder.

    Attributes:
        input_dimension: Input size going into the encoder.
        hidden_layer_size: Number of hidden nodes within each hidden layer of the encoder.
        activation: Non-linear activation method to be used by the encoder.
        hidden_layers: Number of hidden layers within the encoder.
        embedding_dimension: Dimensionality of the final output of the encoder (i.e., the latent space).
        encode: `torch.Sequential` module containing the full encoder.
    """

    def __init__(
        self,
        input_dimension,
        hidden_layer_size=128,
        activation=nn.PReLU,
        hidden_layers=1,
        embedding_dimension=64,
        p_dropout=0.0
    ):
        super().__init__()
        encoder = []
        current_size = input_dimension
        next_size = hidden_layer_size
        for i in range(hidden_layers):
            if i != 0:
                current_size = next_size
                # Slowly halve size of the AE hidden layer dimension
                # over time until we reach the embedding dimension.
                # Take max since we do not want the non-bottleneck
                # layers to become smaller than the bottleneck.
                next_size = max(int(next_size / 2), embedding_dimension)
            encoder.append(nn.Linear(current_size, next_size))
            encoder.append(activation())
            encoder.append(nn.BatchNorm1d(next_size))
            encoder.append(nn.Dropout(p_dropout))
        if hidden_layers > 0:
            encoder.append(nn.Linear(next_size, embedding_dimension))
        else:
            encoder.append(nn.Linear(input_dimension, embedding_dimension))
        # Do not include an activation before the embedding.
        self.encode = nn.Sequential(*encoder)
        self.input_dimension = input_dimension
        self.hidden_layer_size = hidden_layer_size
        self.activation = activation
        self.hidden_layers = hidden_layers
        self.embedding_dimension = embedding_dimension

    def forward(self, x):
        return self.encode(x)


class Decoder(nn.Module):
    """Base class modeling the decoder portion of an autoencoder.

    Attributes:
        decode: `torch.Sequential` module containing the full decoder.
    """

    def __init__(self, encoder, activation=nn.PReLU, p_dropout=0.0):
        super().__init__()
        decoder = []
        # Build up the decoder symmetrically from the encoder.
        for layer in encoder.encode[::-1][::4]:
            current_size = layer.weight.shape[0]
            next_size = layer.weight.shape[1]
            decoder.append(nn.Linear(current_size, next_size))
            decoder.append(activation())
            decoder.append(nn.BatchNorm1d(next_size))
            decoder.append(nn.Dropout(p_dropout))
        self.decode = nn.Sequential(*(decoder[:-3]))

    def forward(self, x):
        return self.decode(x)


class AE(nn.Module):
    """Base class modeling an autoencoder.

    Attributes:
        encode: `torch.Sequential` module containing the full encoder.
        decode: `torch.Sequential` module containing the full decoder.
        input_dimension: Input size going into the encoder.
        hidden_layer_size: Number of hidden nodes within each hidden layer of the encoder.
        activation: Non-linear activation method to be used by the encoder.
        hidden_layers: Number of hidden layers within the encoder.
        embedding_dimension: Dimensionality of the final output of the encoder (i.e., the latent space).
    """

    def __init__(
        self,
        input_dimension,
        hidden_layer_size=128,
        activation=nn.PReLU,
        hidden_layers=1,
        embedding_dimension=64,
    ):
        super().__init__()
        self.encode = Encoder(
            input_dimension,
            hidden_layer_size,
            activation,
            hidden_layers,
            embedding_dimension,
        )
        self.decode = Decoder(self.encode, activation)
        self.input_dimension = input_dimension
        self.hidden_layer_size = hidden_layer_size
        self.activation = activation
        self.hidden_layers = hidden_layers
        self.embedding_dimension = embedding_dimension

    def forward(self, x):
        encoded = self.encode(x)
        decoded = self.decode(encoded)
        return encoded, decoded


class HazardRegression(nn.Module):
    """Base class modeling a cox hazard regression problem.

    Attributes:
        input_dimension: Number of covariates to be input to the cox regression.
        hidden_layer_size: Number of hidden nodes within each hidden layer of the regression.
        activation: Non-linear activation to be used after each hidden layer of the regression.
        hidden_layers: Number of hidden layers to be used within the regression.
    """

    def __init__(
        self,
        input_dimension,
        hidden_layer_size=32,
        activation=nn.PReLU,
        hidden_layers=0,
        p_dropout=0.0
    ):
        super().__init__()
        hazard = []
        current_size = input_dimension
        for layer in range(hidden_layers):
            next_size = hidden_layer_size
            hazard.append(nn.Linear(current_size, next_size))
            hazard.append(activation())
            hazard.append(nn.BatchNorm1d(current_size))
            hazard.append(nn.Dropout(p_dropout))
            current_size = next_size
        hazard.append(nn.Linear(current_size, 1, bias=False))
        self.hazard = nn.Sequential(*hazard)

    def forward(self, x):
        return self.hazard(x)


class SupervisedCoxAE(nn.Module):
    def __init__(self, input_dimension):
        super().__init__()
        self.ae = AE(input_dimension)
        self.hazard = HazardRegression(64)

    def forward(self, x):
        encoded, decoded = self.ae(x)
        hazard = self.hazard(encoded)
        return hazard, x, decoded

In [5]:
from sksurv.datasets import load_flchain

In [6]:
data = load_flchain()

In [7]:
X = data[0].dropna()[["age", "creatinine", "kappa"]]
target = data[1][X.index]
target = np.array([f"{int(i[0])}|{i[1]}" for i in target])
from skorch.net import NeuralNet

class BaseSurvivalNeuralNet(NeuralNet):
    def score(self, X: np.ndarray, y: np.ndarray):
        try:
            concordance = concordance_index(
                event_times=np.vstack(np.char.split(np.array(y), sep="|"))[
                    :, 1
                ].astype(np.float32),
                predicted_scores=np.negative(np.squeeze(self.predict(X))),
                event_observed=np.vstack(np.char.split(np.array(y), sep="|"))[
                    :, 0
                ].astype(int),
            )
        except ValueError:
            concordance = np.nan
        return concordance
    
    def get_loss(self, y_pred, y_true, X=None, training=False):
        return self.criterion_(y_pred, y_true)
    

In [141]:
net = BaseSurvivalNeuralNet(
    module=SupervisedCoxAE,
    module__input_dimension=X.shape[1],
    criterion=ae_criterion,
    lr=0.001
)
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
pipe = make_pipeline(StandardScaler(), net)
net.fit(X.to_numpy().astype(np.float32), target)
net.score(
    X.to_numpy().astype(np.float32), target
)

  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1     1688.5489     2429.6079  0.1824
      2     1640.7899     2154.3174  0.0530
      3     1562.3875     2028.8997  0.0561
      4     1394.7387     2379.1687  0.0609
      5     1048.3701     3100.4685  0.0679
      6      555.0559     2764.8933  0.0674
      7      196.5069      804.6316  0.0641
      8       70.1939      391.5276  0.0540
      9       41.8473      456.0356  0.0510
     10       35.7961      387.7709  0.0485


0.5892330985472448

In [8]:
target

array(['1|85.0', '1|1281.0', '1|69.0', ..., '1|51.0', '1|611.0',
       '1|2188.0'], dtype='<U8')

In [81]:
import numpy as np
from lifelines.utils import concordance_index
from skorch.net import NeuralNet
from sksurv.linear_model.coxph import BreslowEstimator
from torch import nn
from itertools import chain
from operator import add
import itertools
from collections.abc import Iterable

def flatten(l):
    for el in l:
        if isinstance(el, Iterable) and not isinstance(el, (str, bytes)):
            yield from flatten(el)
        else:
            yield el


def transform_survival_target(time, event):
    return np.array([f"{time[i]}|{event[i]}" for i in range(len(time))])

def inverse_transform_survival_target(y):
    return (
        np.array([float(i.rsplit("|")[0])for i in y]),
        np.array([int(i.rsplit("|")[1])for i in y])
    )

def inverse_transform_survival_function(y):
    return np.vstack(
        [np.array(i.rsplit("|")) for i in y]
    )

class BaseSurvivalNeuralNet(NeuralNet):
    def get_loss(self, y_pred, y_true, X=None, training=False):
        return self.criterion_(y_pred, y_true)


class HazardRegression(nn.Module):
    def __init__(
        self,
        input_dimension,
        n_output,
        hidden_layer_sizes,
        activation=nn.ReLU,
        p_dropout=0.0,
    ) -> None:
        super().__init__()
        hidden_layer_sizes = hidden_layer_sizes + [n_output]
        print(hidden_layer_sizes)
        self.activation = activation
        self.p_dropout = p_dropout
        self.hidden_layer_sizes = hidden_layer_sizes
        self.n_output = n_output
        huh = list(flatten([
                nn.Linear(input_dimension, hidden_layer_sizes[0]),
                nn.ReLU(),
                nn.Dropout(p_dropout),
            ]
            + [
                [
                    nn.Linear(
                        hidden_layer_sizes[i], hidden_layer_sizes[i + 1]
                    ),
                    nn.ReLU(),
                    nn.Dropout(p_dropout),
                ]
                for i in range(len(hidden_layer_sizes) -1)
            ]))
        self.hazard = nn.Sequential(*flatten(
        [
                nn.Linear(input_dimension, hidden_layer_sizes[0]),
                nn.ReLU(),
                nn.Dropout(p_dropout),
            ]
            + [
                [
                    nn.Linear(
                        hidden_layer_sizes[i], hidden_layer_sizes[i + 1]
                    ),
                    nn.ReLU(),
                    nn.Dropout(p_dropout),
                ]
                for i in range(len(hidden_layer_sizes) -1)
            ]
        ))

In [82]:
hm = HazardRegression(100, 1, [200, 100], nn.ReLU, 0.5)

[200, 100, 1]


In [89]:
hm.hazard[0].weight.shape

torch.Size([200, 100])

In [90]:
hm

HazardRegression(
  (hazard): Sequential(
    (0): Linear(in_features=100, out_features=200, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=200, out_features=100, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.5, inplace=False)
    (6): Linear(in_features=100, out_features=1, bias=True)
    (7): ReLU()
    (8): Dropout(p=0.5, inplace=False)
  )
)

In [98]:
torch.sum(hm.hazard[1:].parameters())

TypeError: sum(): argument 'input' (position 1) must be Tensor, not generator

In [108]:
hu = [i for i in hm.hazard[1:].weight()][0]

AttributeError: 'Sequential' object has no attribute 'weight'

In [131]:
a = torch.sum(torch.cat([torch.norm(i) for i in hm.hazard[1:].parameters()]))

RuntimeError: zero-dimensional tensor (at position 0) cannot be concatenated

In [140]:
hm.hazard[0].weight[:, [0, 1, 2, 3]].shape

torch.Size([200, 4])

In [128]:
dir(a)

['T',
 '__abs__',
 '__add__',
 '__and__',
 '__array__',
 '__array_priority__',
 '__array_wrap__',
 '__bool__',
 '__class__',
 '__complex__',
 '__contains__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__div__',
 '__dlpack__',
 '__dlpack_device__',
 '__doc__',
 '__eq__',
 '__float__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__idiv__',
 '__ifloordiv__',
 '__ilshift__',
 '__imod__',
 '__imul__',
 '__index__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__irshift__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__long__',
 '__lshift__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__nonzero__',
 '__or__',
 '__pos__',
 '__pow__',
 '__radd__',
 '__rand__',
 '__rdiv__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '

In [144]:
len([1, 2, 3]) * torch.tensor(5., requires_grad=True)

tensor(15., grad_fn=<MulBackward0>)